# Diffusion in small-world networks with reflexivity

This code reproduces the diffusion model present in

*Diffusion dynamics in small-world networks with heterogeneous consumers* from Delre, Sebastiano A., Jager, Wander and Janssen, Marco A., Computational and Mathematical Organization Theory, **13**, 2, 2007.

In [128]:
# Necessary libraries

from __future__ import division

%matplotlib qt

import matplotlib.pyplot as plt
import matplotlib.animation as animation

import networkx as nx
import numpy as np

## Auxiliary functions

In [ ]:
def get_neighbors(graph, node, level):
    """Get neighbors of a given node up to a certain level"""
    # All neighbors up to the given level
    all_neighbors = nx.single_source_shortest_path_length(graph, node, cutoff=level)
    
    # Select only neighbors at the level we want
    return [k for (k, v) in all_neighbors.items() if v == level]

In [ ]:
def set_colors(graph):
    """
    Create a list of colors to apply to graph
    
    Adopters are blue and non-adopters are red
    """
    colors = []
    for n in graph.nodes():
        node = G.node[n]
        if node['adopter'] == 1:
            colors.append('b')
        else:
            colors.append('r')
    
    return colors

In [98]:
def is_adopter(graph, node):
        """Return True if a node is an adopter"""
        return graph.node[node]['adopter'] == 1

In [202]:
def draw_graph(graph, node_positions):
    """Function to draw the graph in which the evolution is occurring"""
    nx.draw_networkx_nodes(graph, node_positions, node_color=set_colors(graph), node_size=50)
    nx.draw_networkx_edges(graph, node_positions, width=0.3, alpha=0.5)

In [203]:
def animate(i, graph, node_positions, parameters, test=False):
    """Function to animate the algorithm evolution"""
    #print(i)
    if test:
        node = graph.node[i]
        node['adopter'] = 1
    else:
        evolution_step(graph, parameters)
    draw_graph(graph, node_positions)

In [31]:
def speed_of_difussion(adopters):
    """
    Compute the velocity of diffusion $rho$
    
    This is done according to equation 6 in the article
    """
    T = len(adopters)
    cumulative_adopters = [ sum(adopters[:i]) for i in range(1, T+1) ]
    return (1/T) * ( sum(cumulative_adopters) / sum(adopters) )

## Algorithm

In [255]:
def generate_initial_conditions(number_of_nodes, parameters):
    """
    Initial conditions for the simulation
    
    Create the graph on which the difussion occurs and set additional
    attributes for its node
    
     `parameters` is a dictionary that contains the parameters that control
     the evolution.
    """
    # Network creation.
    # 4 is the average number of neighbors each node has in the graph
    # This number is fixed in the article
    G = nx.generators.watts_strogatz_graph(number_of_nodes, 4,
                                           parameters['randomness'])
    
    for n in G.nodes():
        node = G.node[n]
        node['adopter'] = 0                              # 1 is adopter, 0 non-adopter
        node['preference'] = np.random.random()          # pi
        
        # Consumers at level 2
        node['neighbors_level_2'] = get_neighbors(G, n, level=parameters['level'])
        
        # Total number of neighbors up to level 2
        node['number_of_neighbors'] = len(G.neighbors(n) + node['neighbors_level_2'])
    
    return G

In [46]:
def evolution_step(graph, parameters):
    """
    Function that computes the evolution step of the difussion process
    that occurs in a small-world graph
    
    `parameters` is a dictionary that contains the parameters that
    control the evolution.
    """
    
    marketing_effort = parameters['marketing_effort']
    social_threshold = parameters['social_threshold']
    social_influence = parameters['social_influence']
    minimal_utility = parameters['minimal_utility']
    
    # Adopters before performing the current step
    previous_adopters = [x for x in graph.nodes() if is_adopter(graph, x)]
    
    for n in graph.nodes():
        node = graph.node[n]

        # -- Adoption due to marketing
        p = np.random.random()
        if not node['adopter'] and (p < marketing_effort):
            node['adopter'] = 1

        # -- Adoption due to utility

        # Adopters at level 1
        adopters_level_1 = [x for x in graph.neighbors(n) if is_adopter(graph, x)]

        # Adopters at level 2
        adopters_level_2 = [x for x in node['neighbors_level_2'] if is_adopter(graph, x)]

        # Total number of adopters
        adopters_among_neighbors = len(adopters_level_1) + len(adopters_level_2)

        # Only if a consumer has adopters among his neighbors, he decides to adopt!
        if adopters_among_neighbors > 0:

            # Ai value
            adopters_percentaje = adopters_among_neighbors / node['number_of_neighbors']

            # Computing xi
            if adopters_percentaje > social_threshold:
                local_influence = 1
            else:
                local_influence = 0
            
            # Set individual preference (yi)
            if parameters['quality'] >= node['preference']:
                individual_preference = 1
            else:
                individual_preference = 0

            # Computing utility Ui
            utility = social_influence * local_influence + (1 - social_influence) * individual_preference

            # print(utility)
            if utility > minimal_utility:
                if np.random.random() > 1 - minimal_utility:
                    node['adopter'] = 1
    
    # Return number of adopters at time t
    current_adopters = [x for x in graph.nodes() if is_adopter(graph, x)]
    return len(current_adopters) - len(previous_adopters)

In [ ]:
def evolution(max_time, graph, parameters):
    """Compute the evolution of the algorithm up to max_time"""
    # Save the adopters at each time during the evolution
    adopters = []
    
    # Print the parameters of the run
    print(parameters)
    
    # Perform the evolution
    for t in range(t):
        n = evolution_step(G, parameters)
        adopters.append(n)
    
    return adopters

## Running the algorithm

These are main parameters that control the evolution of the algorithm, and their corresponding variable in the article:

* Network randomness (`randomness`): $r$
* Quality: $q_{j}$
* Marketing effort: $e_{1}$
* Social influence: $\beta_{j}$
* Social threshold: $h_{i}$
* Minimal utility:  $U_{i, MIN}$

In [246]:
parameters = dict(
    randomness = 0.01,
    quality = 0.5,
    marketing_effort = 0.1,
    social_influence = 1,
    social_threshold = 0.3,
    minimal_utility = 0.9,
    level = 2,
)

In [256]:
G = generate_initial_conditions(number_of_nodes=50, paremeters=paremeters)

In [56]:
fig = plt.figure()
positions = nx.spring_layout(G)
animation.FuncAnimation(fig, lambda i: animate(i, G, positions, test=True), frames=50, interval=1, repeat=False)